In [1]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.utils.data

from sklearn.mixture import GaussianMixture 
from scipy import signal

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def loadData(file):
    data = pd.read_csv(file,infer_datetime_format=True, parse_dates=['Date'])
    print('Raw shape: ',data.shape)
    data['date'] = data.Date #pd.to_datetime(data.date)
    data = data[ (data.Date < pd.to_datetime('2019')) & (data.Date > pd.to_datetime('2017-07')) ]
    print('Days: ',len(set(data.date)))
    return data

In [3]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='amount', index=['date'],
                    columns=['start_id','end_id'], aggfunc=np.sum, fill_value=0)
    return table,table.index.values

In [4]:
def getMatrixData():
    file = dataDir + dataFile
    dataRaw = loadData(file)
    dataTs,dates = getTimeSeries(dataRaw)
    matrix = np.load('communityAggregatedMatrix.npy')
    print('Matrix Shape: ',matrix.shape)
    return matrix,dates

In [5]:
dataDir = '/home/urwa/Documents/Projects/AnomalyDetection/Pipeline/data/'
dataFile = 'NewYorkEdgeDatewiseAggregated.csv'
#events_data =dataDir+'TaipeiEvents.csv'

In [6]:
#matrix = np.stack((dataOut.values, dataIn.values),-1)
matrix,dates = getMatrixData()
matrix = matrix.astype(float)
matrix.shape

Raw shape:  (21380943, 4)
Days:  548
Matrix Shape:  (548, 576)


(548, 576)

In [7]:
min(dates), max(dates)

(numpy.datetime64('2017-07-02T00:00:00.000000000'),
 numpy.datetime64('2018-12-31T00:00:00.000000000'))

In [110]:
# matrix = signal.detrend(matrix, axis =1)

In [111]:
# for i in range(matrix.shape[1]):
#         matrix[:, i] = (matrix[:, i] - np.mean(matrix[:, i])) / (np.std(matrix[:, i]))

In [112]:
for i in range(matrix.shape[0]):
        matrix[i,:] = (matrix[i,:] - np.mean(matrix[i,:])) / (np.std(matrix[i,:]))

In [113]:
# stations = list(set(dataRaw.start_id))
# n= len(stations)
# edge_index = [[a//n,a%n] for a in range(n*n)]
# edge_index = torch.tensor(edge_index, dtype=torch.long)

In [114]:
#dates = list(dataOut.index)
DOW = list(pd.to_datetime(dates).dayofweek)
DOW = ((np.array(DOW) == 5) | (np.array(DOW) == 6)).astype(int)
DOW[:10]

array([1, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [115]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(input_dim, hiddenDim[0])
        self.linear2 = nn.Linear(hiddenDim[0], hiddenDim[1])
        self.linear3 = nn.Linear(hiddenDim[1], hiddenDim[2])
        self.linear4 = nn.Linear(hiddenDim[2], n_classes)

        self.init_weights(self.linear1)
        self.init_weights(self.linear2)
        self.init_weights(self.linear3)
        self.init_weights(self.linear4)
        
    def forward(self, x):
        #x, edge_index = data.x, data.edge_index
        x = self.linear1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear4(x)
        return F.log_softmax(x, dim=1)
        
    def init_weights(self,m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
    
    def representation(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        return x
            

In [116]:
matrixtensor = torch.tensor(matrix).float()

In [117]:
input_dim = matrix.shape[1]

In [118]:
hiddenDim = [500,100,20]
n_classes = 2

learning_rate=0.001
batch_size = 50
num_epochs = 200

In [119]:
Y = torch.tensor(np.array(DOW), dtype=torch.long)

In [120]:
matrixtensor.shape

torch.Size([548, 576])

In [121]:
data_tensor = torch.utils.data.TensorDataset(matrixtensor, Y) 
dataloader = torch.utils.data.DataLoader(dataset = data_tensor, batch_size = batch_size, shuffle = True)

In [122]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model = Net().to(device)
#data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [123]:
for epoch in range(num_epochs):
    for data in dataloader:
        X, y = data
        X = X.cuda()
        y = y.cuda()
#         print(X.shape)
#         print(y.shape)
        # ===================forward=====================
        output = model(X)
        #print(output.shape)
        loss = criterion(output, y)
        #MSE_loss = nn.MSELoss()(output, y)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
        .format(epoch + 1, num_epochs, loss.item()))

epoch [1/200], loss:0.7595
epoch [2/200], loss:0.6322
epoch [3/200], loss:0.6193
epoch [4/200], loss:0.4831
epoch [5/200], loss:0.5302
epoch [6/200], loss:0.4459
epoch [7/200], loss:0.4605
epoch [8/200], loss:0.5711
epoch [9/200], loss:0.4906
epoch [10/200], loss:0.3328
epoch [11/200], loss:0.2923
epoch [12/200], loss:0.3878
epoch [13/200], loss:0.2618
epoch [14/200], loss:0.1971
epoch [15/200], loss:0.2940
epoch [16/200], loss:0.2959
epoch [17/200], loss:0.2709
epoch [18/200], loss:0.2436
epoch [19/200], loss:0.2154
epoch [20/200], loss:0.2034
epoch [21/200], loss:0.2511
epoch [22/200], loss:0.2609
epoch [23/200], loss:0.2260
epoch [24/200], loss:0.2310
epoch [25/200], loss:0.2219
epoch [26/200], loss:0.1502
epoch [27/200], loss:0.1853
epoch [28/200], loss:0.1432
epoch [29/200], loss:0.0934
epoch [30/200], loss:0.1392
epoch [31/200], loss:0.1720
epoch [32/200], loss:0.1710
epoch [33/200], loss:0.0642
epoch [34/200], loss:0.2385
epoch [35/200], loss:0.1486
epoch [36/200], loss:0.1661
e

In [124]:
model.eval()
with torch.no_grad():
    y = np.array(DOW)
    predList = model(matrixtensor.cuda()).cpu().numpy()
    pred = np.argmax(predList,axis=1)    
    print('Accuracy: {:.4f}'.format( np.sum(pred == y) / pred.shape[0] ))

Accuracy: 0.9982


In [125]:
with torch.no_grad():
    representation = model.representation(matrixtensor.cuda()).cpu().numpy()
representation.shape

(548, 20)

In [8]:
# import events data
events_data =dataDir+'NewYorkEvents.csv'
df_events = pd.read_csv(events_data, encoding = "ISO-8859-1", parse_dates=['Date'], infer_datetime_format=True)


In [9]:
df_events.head()

,Type,Name,Date
0,National Holiday,New Year's Day,2016-01-01
1,National Holiday,"Martin Luther King, Jr. Day",2016-01-18
2,National Holiday,Presidents' Day,2016-02-15
3,National Holiday,Memorial Day,2016-05-30
4,National Holiday,Independence Day,2016-07-04


In [10]:
holidayDates = df_events[df_events.Type == 'National Holiday'].Date
len(holidayDates)

41

In [12]:
holidayDates = [str(d.date()) for d in holidayDates]
dates = pd.to_datetime(dates)
dates = [str(d.date()) for d in dates]

In [13]:
anomalyIndex = [i for i,d in enumerate(dates) if d in holidayDates]
len(anomalyIndex)
indexBool = np.array([i in anomalyIndex for i in list(range(matrix.shape[0]))])

In [15]:
sum(indexBool), len(anomalyIndex)

(16, 16)

In [131]:
def anomalyDetection(y,pval = 0.2,iterN=5,n_com=1):
    #index of regular (non-outlier points)
    #rind=y[:,0]>-10 
    rind = np.array(range(y.shape[0]))
    
    #clustering model
    gm=GaussianMixture(n_components=n_com, n_init=100, max_iter=1000,random_state=0) 
    for i in range(iterN): #iterate
        print('Iteration {}'.format(i+1))  
        clustering=gm.fit(y[rind,:]) #fit EM clustering model excluding outliers
        l=clustering.score_samples(y) #estimate likelihood for each point
        Lthres=sorted(l)[int(len(l)*pval)] #anomaly threshold
        rind0=0+rind
        rind=l>Lthres #non-anomalous points
        if(sum(rind)==0):
            print('All anomalies in {} iterations'.format(i+1))
            break
        if all(rind==rind0):
            print('Convergence in {} iterations'.format(i+1))
            break
    return l < Lthres

In [132]:
def getResults(reducedMatrix,threshHolds,iterN=5,n_com=1):
    results = []
    for th in threshHolds:
        #th = thres/100
        print("Threshhold: ",th)
        outliers = anomalyDetection(reducedMatrix,th,iterN,n_com)

        tpr = sum(outliers & indexBool)/sum(indexBool)
        fpr = sum(outliers & ~indexBool)/sum(~indexBool)
        precision = sum(outliers & indexBool)/sum(outliers)

        F1 = 2 * (precision * tpr) / (precision + tpr)

        res = {'Cat':'Global', 'th':th, 'TPR':tpr, 'FPR':fpr, 'F1':F1, 'Precision':precision}
        results.append(res)

    resDf = pd.DataFrame(results)    
    return resDf

In [133]:
# f,ax = plt.subplots(representation.shape[1],1,figsize=(20, 60))
# for i in range(representation.shape[1]):
#     ax[i].plot(pd.to_datetime(dates), representation[:,i])
#     ax[i].plot(pd.to_datetime(dates), signal.detrend(representation[:,i]))

In [134]:
threshHolds = [0.01, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [135]:
Res1 = getResults(representation,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.05
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.5
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.6
Iteratio

In [136]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.380952,0.001880,0.800000,0.2500,0.01
1,Global,0.562500,0.013158,0.562500,0.5625,0.03
2,Global,0.486486,0.022556,0.428571,0.5625,0.04
3,Global,0.511628,0.030075,0.407407,0.6875,0.05
4,Global,0.458333,0.039474,0.343750,0.6875,0.06
5,Global,0.407407,0.050752,0.289474,0.6875,0.07
6,Global,0.372881,0.060150,0.255814,0.6875,0.08
7,Global,0.314286,0.080827,0.203704,0.6875,0.10
8,Global,0.192000,0.182331,0.110092,0.7500,0.20
9,Global,0.133333,0.285714,0.073171,0.7500,0.30


In [137]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca.fit(matrix)

PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [138]:
reducedMatrixPCA = pca.transform(matrix)
print(reducedMatrixPCA.shape)

(548, 20)


In [139]:
# f,ax = plt.subplots(reducedMatrixPCA.shape[1],1,figsize=(20, 60))
# for i in range(reducedMatrixPCA.shape[1]):
#     ax[i].plot(pd.to_datetime(dates), reducedMatrixPCA[:,i])
#     ax[i].plot(pd.to_datetime(dates), signal.detrend(reducedMatrixPCA[:,i]))

In [140]:
Res1 = getResults(reducedMatrixPCA,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.05
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.5
Iteration 1
Iteration

In [141]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.285714,0.003759,0.600000,0.1875,0.01
1,Global,0.562500,0.013158,0.562500,0.5625,0.03
2,Global,0.540541,0.020677,0.476190,0.6250,0.04
3,Global,0.511628,0.030075,0.407407,0.6875,0.05
4,Global,0.458333,0.039474,0.343750,0.6875,0.06
5,Global,0.407407,0.050752,0.289474,0.6875,0.07
6,Global,0.372881,0.060150,0.255814,0.6875,0.08
7,Global,0.342857,0.078947,0.222222,0.7500,0.10
8,Global,0.192000,0.182331,0.110092,0.7500,0.20
9,Global,0.133333,0.285714,0.073171,0.7500,0.30
